# langchain学习

安装python包

In [11]:
%pip install -U langchain-community langgraph langchain-anthropic tavily-python

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
ERROR: Ignored the following versions that require a different python version: 0.0.1 Requires-Python >=3.8.1,<4.0; 0.0.10 Requires-Python >=3.8.1,<4.0; 0.0.11 Requires-Python >=3.8.1,<4.0; 0.0.12 Requires-Python >=3.8.1,<4.0; 0.0.13 Requires-Python >=3.8.1,<4.0; 0.0.14 Requires-Python >=3.8.1,<4.0; 0.0.15 Requires-Python >=3.8.1,<4.0; 0.0.16 Requires-Python >=3.8.1,<4.0; 0.0.17 Requires-Python >=3.8.1,<4.0; 0.0.18 Requires-Python >=3.8.1,<4.0; 0.0.19 Requires-Python >=3.8.1,<4.0; 0.0.1rc1 Requires-Python >=3.8.1,<4.0; 0.0.1rc2 Requires-Python >=3.8.1,<4.0; 0.0.2 Requires-Python >=3.8.1,<4.0; 0.0.20 Requires-Python >=3.8.1,<4.0; 0.0.21 Requires-Python >=3.8.1,<4.0; 0.0.22 Requires-Python >=3.8.1,<4.0; 0.0.23 Requires-Python >=3.8.1,<4.0; 0.0.24 Requires-Python >=3.8.1,<4.0; 0.0.25 Requires-Python >=3.8.1,<4.0; 0.0.26 Requires-Python >=3.8.1,<4.0; 0.0.27 Requires-Python >=3.8.1,<4.0; 0.0.28 Requires-Python >=3.8.1,<4.0; 0.0.29 

### 一、切割文档用于制作知识库

In [2]:
# Import relevant functionality
# 导入ChatAnthropic类，用于创建具有人类般推理能力的聊天机器人
from langchain_anthropic import ChatAnthropic
from langchain_community.chat_models import ChatTongyi
# 导入TavilySearchResults类，用于处理Tavily搜索算法的结果
from langchain_community.tools.tavily_search import TavilySearchResults
# 导入HumanMessage类，用于表示人类用户的输入消息
from langchain_core.messages import HumanMessage
# 导入SqliteSaver类，用于将聊天记录保存到SQLite数据库
from langgraph.checkpoint.sqlite import SqliteSaver
# 导入create_react_agent函数，用于创建一个反应式代理
from langgraph.prebuilt import create_react_agent

# Create the agent
memory = SqliteSaver.from_conn_string(":memory:")
# model = ChatAnthropic(model_name="claude-3-sonnet-20240229")
model = ChatTongyi(model_name="qwen1.5-1.8b-chat")
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

# Use the agent
config = {"configurable": {"thread_id": "abc123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob! and i live in sf")]}, config
):
    print(chunk)
    print("----")

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hello Bob! Nice to meet you in San Francisco! What brings you here? \n\nSan Francisco is a vibrant and diverse city known for its rich history, stunning architecture, iconic landmarks, and a thriving arts and culture scene. As an AI language model, I'm well-equipped to provide information on various aspects of the city's culture, attractions, and popular neighborhoods.\n\nIn terms of cultural activities, San Francisco offers a wide range of options. Here are some must-visit places:\n\n1. Golden Gate Bridge: One of the most recognizable landmarks in the world, the Golden Gate Bridge is a suspension bridge that spans the Golden Gate Strait, connecting San Francisco Bay with Marin County. You can take a walk across the bridge or explore the surrounding parks like Golden Gate Park, where you'll find gardens, museums, and many recreational opportunities.\n\n2. Alcatraz Island: This former federal prison was converted into a National Park and has be

### 二、设置emmbeding层，用于数据的向量化，存入数据库

In [6]:
from langchain_community.embeddings import DashScopeEmbeddings
embedding_name = r"text-embedding-v1"
ds_embeddings = DashScopeEmbeddings(model=embedding_name)


### 三、保存向量到数据库

In [9]:
from langchain_community.vectorstores import Chroma
import sentence_transformers

# # 初始化数据库
# db = Chroma.from_documents(split_docs, ds_embeddings,persist_directory="./chroma/news_test")
# # 持久化
# db.persist()
# 对数据进行加载
db = Chroma(persist_directory="./chroma/news_test", embedding_function=ds_embeddings)



### 四、测试知识库是否可用

In [12]:
question = "三打白骨精"
# 寻找四个相似的样本
similarDocs = db.similarity_search(question,k=4)
print(similarDocs)


[Document(page_content='为至宝，怎肯轻与你这粉骷髅。”他两个散言碎语的，直斗到更深，唐长老全不动念。那女怪扯扯拉拉的不放，这师父只是老老成成的不肯。直缠到有半夜时候，把那怪弄得恼了，叫：“小的们，拿绳来！”可怜将一个心爱的人', metadata={'source': '西游记.txt'}), Document(page_content='的，若不做个威风，他怎肯服降哩？”陡然间，就狐假虎威，红须倒竖，血发朝天，眼睛迸裂，大喝一声道：“带那和尚进来！”众妖们，大家响响的答应了一声“是！”就把三藏望里面只是一推。这是既在矮檐下，怎敢不低头', metadata={'source': '西游记.txt'}), Document(page_content='！”那妖精就举铲来敌。这一场，从天井中斗出前门。看他三僧攒一怪！好杀：', metadata={'source': '西游记.txt'}), Document(page_content='打杀了，拖在门口骂战哩。”那老妖闻言，愈加烦恼道：“这厮却也无知！我倒不曾吃他师父，他转打杀我家先锋，可恨！可恨！”叫：“取披挂来。我也只闻得讲甚么孙行者，等我出去，看是个甚么九头八尾的和尚，拿他进来', metadata={'source': '西游记.txt'})]


### 五、调用大模型来结合数据库来问答

In [23]:

from langchain.chains import RetrievalQA
from langchain_community.llms import Tongyi

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=Tongyi(), retriever=retriever)

query = "三打白骨精是第几章？"
print(qa.run(query))


 三打白骨精是上卷的第二十七回。


### 注：
 教程采用了通义千问的emmbeding层和大模型，需要到阿里云上根据教程设置调用的key
 链接：https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key?spm=a2c4g.11186623.0.i21